In [1]:
from pathlib import Path
import sys

import pandas as pd
import numpy as np

sys.path.append('../')
import payulator as pl


%load_ext autoreload
%autoreload 2

In [2]:
# Test some

principal = 11000
interest = 0.099
compounding_freq = 'monthly'
payment_freq = 'monthly'
num_payments = 3*12
fee = 350
start_date = '2018-03-06'

A = pl.compute_amortized_loan(principal, interest, compounding_freq, payment_freq, num_payments, 
  fee, start_date)

A


{'interest_and_fee_total': 2109.22,
 'interest_and_fee_total/principal': 0.19,
 'interest_total': 1759.22,
 'payment_schedule':     payment_sequence payment_date  beginning_balance  principal_payment  \
 0                  1   2018-03-06           11000.00             263.67   
 1                  2   2018-04-06           10736.33             265.85   
 2                  3   2018-05-06           10470.48             268.04   
 3                  4   2018-06-06           10202.44             270.25   
 4                  5   2018-07-06            9932.18             272.48   
 5                  6   2018-08-06            9659.70             274.73   
 6                  7   2018-09-06            9384.97             277.00   
 7                  8   2018-10-06            9107.98             279.28   
 8                  9   2018-11-06            8828.69             281.59   
 9                 10   2018-12-06            8547.11             283.91   
 10                11   2019-01-06   

In [3]:
# Test some

principal = 30000
interest = 0.1
payment_freq = 'monthly'
num_payments = 13
fee = 0
start_date = '2018-04-06'

A = pl.compute_interest_only_loan(principal, interest, payment_freq, num_payments, 
  fee, start_date)

A


{'interest_and_fee_total': 3250.0,
 'interest_and_fee_total/principal': 0.11,
 'interest_total': 3250.0,
 'payment_schedule':     payment_sequence payment_date  beginning_balance  principal_payment  \
 0                  1   2018-04-06              30000                  0   
 1                  2   2018-05-06              30000                  0   
 2                  3   2018-06-06              30000                  0   
 3                  4   2018-07-06              30000                  0   
 4                  5   2018-08-06              30000                  0   
 5                  6   2018-09-06              30000                  0   
 6                  7   2018-10-06              30000                  0   
 7                  8   2018-11-06              30000                  0   
 8                  9   2018-12-06              30000                  0   
 9                 10   2019-01-06              30000                  0   
 10                11   2019-02-06     

In [4]:
principal = 11000
interest = 0.099
compounding_freq = 'monthly'
payment_freq = 'monthly'
num_years = 2
fee = 350
start_date = '2018-03-06'

A = pl.compute_amortized_loan(principal, interest, compounding_freq, payment_freq, num_years, 
  fee, start_date)
B = pl.compute_amortized_loan(principal, interest, compounding_freq, 'weekly', num_years, 
  fee, '2019-03-07')

g = pl.aggregate_payment_schedules([A['payment_schedule'], B['payment_schedule']])
g

,payment_date,principal_payment,interest_payment,total_payment
0,2018-03-06,5477.41,90.75,5568.16
1,2018-04-06,5522.59,45.56,5568.15
2,2019-03-07,5494.79,20.88,5515.67
3,2019-03-14,5505.21,10.45,5515.66


In [5]:
# Someday/maybe create a Loan class and include ``loan_code`` as an attribute.
# Then log the contributing loan codes in a new column of ``aggregate_payment_schedules``